In [1]:
import pandas as pd

import os 
import glob 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.layers import *
from tensorflow.keras.applications import VGG16 as vgg16
from tensorflow.keras.preprocessing import image as image_utils 
from tensorflow.keras.applications.vgg16 import preprocess_input

In [2]:
df = pd.read_csv('C:/Users/user/Documents/빅데이터/11주차/archive/pokemon.csv')
df

,Name,Type1,Type2
0,bulbasaur,Grass,Poison
1,ivysaur,Grass,Poison
2,venusaur,Grass,Poison
3,charmander,Fire,NaN
4,charmeleon,Fire,NaN
...,...,...,...
804,stakataka,Rock,Steel
805,blacephalon,Fire,Ghost
806,zeraora,Electric,NaN
807,meltan,Steel,NaN


# 이미지 경로 정의

In [3]:
# 포켓몬 이미지 파일 경로
PATH = 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images'
# 포켓몬 이미지 파일 경로 리스트화
PATHS = glob.glob(os.path.join(PATH, "*"))
print(PATHS)

['C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\abomasnow.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\abra.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\absol.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\accelgor.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\aegislash-blade.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\aerodactyl.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\aggron.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\aipom.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\alakazam.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\alomomola.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\altaria.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\amaura.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/images\\ambipom.png', 'C:/Users/user/Documents/빅데이터/11주차/archive/images/image

# VGG를 위한 이미지 처리 함수 정의

In [4]:
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 

# 이미지 출력함수
def show_image(image_path):
    image = mpimg.imread(image_path)
# print(image.shape)
    plt.imshow(image)
    

# 이미지 처리를 위한 함수
# 이미지 경로 입력
def load_and_process_image(image_path): 
    # 이미지 로드, 사이즈(224, 224)
    image = image_utils.load_img(image_path, target_size=(224, 224))
    # 이미지 -> 넘파이 배열 형태
    image = image_utils.img_to_array(image)
    # 한 차원 확장. 사이즈 (224, 224, 3) -> (1,224,224,3)
    image = image.reshape(1,224,224,3)
    # 이미지 정규화
    image = preprocess_input(image)
    return image

# 넘파이로 코사인 유사도 구하는 함수
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

# 학습된 모델 로드

In [5]:
# 입력 이미지 크기
inputs = keras.Input(shape=(224, 224, 3))
# 사전 학습된 vgg16 불러오기
feature = vgg16(include_top=False,
                weights='imagenet')(inputs)
# 평균 특징
flatten = keras.layers.GlobalAveragePooling2D()(feature)
# 하나의 모델로 묶기
model = keras.Model(inputs, flatten)

In [6]:
# 모델 입력, 출력 확인
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
Total params: 14714688 (56.13 MB)
Trainable params: 14714688 (56.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# 메인 프로그램

In [7]:
# 유사도 결과값 리스트
result = []

# 포켓몬 유사도 구하는 메인 프로그램
# 포켓몬 경로를 리스트로 입력받기
def pokemon(image_path, others_image_path):
    # 이미지 정의
    image = load_and_process_image(image_path)
    # vgg16으로 이미지 특징 추출 후, 1차원 감소
    target = model.predict(image).squeeze()

    # 결과값 리스트 초기화
    result.clear()
    
    # 다른 이미지와 코사인 유사도 측정
    for others_path in others_image_path:
        # 다른 이미지 정의
        other_image = load_and_process_image(others_path)
        # 다른 이미지 특징 추출 후, 1차원 감소
        now = model.predict(other_image).squeeze()
        
        # 두 차원의 코사인 유사도를 계산
        cosine = cosine_similarity(target , now)

        # 결과값 리스트에 저장
        result.append(f"{cosine}")
        
        # 코사인 유사도 출력
        print (f"{image_path.split('/')[-1] } and {others_path.split('/')[-1]} : {cosine}")

In [8]:
# 포켓몬 번호 리스트
pokemon_num = []

for i in range(0,809):
    pokemon_num.append(i)

# 포켓몬 이름 리스트
pokemon_name = []

for num in range(len(os.listdir('C:/Users/user/Documents/빅데이터/11주차/archive/images/images'))):
    pokemon_name.append(os.listdir('C:/Users/user/Documents/빅데이터/11주차/archive/images/images')[num].split('.')[0])

# 포켓몬 번호, 이름 딕셔너리
pokemon_list= dict(zip(pokemon_num, pokemon_name))

# 포켓몬 딕셔너리
print(pokemon_list)

{0: 'abomasnow', 1: 'abra', 2: 'absol', 3: 'accelgor', 4: 'aegislash-blade', 5: 'aerodactyl', 6: 'aggron', 7: 'aipom', 8: 'alakazam', 9: 'alomomola', 10: 'altaria', 11: 'amaura', 12: 'ambipom', 13: 'amoonguss', 14: 'ampharos', 15: 'anorith', 16: 'araquanid', 17: 'arbok', 18: 'arcanine', 19: 'arceus', 20: 'archen', 21: 'archeops', 22: 'ariados', 23: 'armaldo', 24: 'aromatisse', 25: 'aron', 26: 'articuno', 27: 'audino', 28: 'aurorus', 29: 'avalugg', 30: 'axew', 31: 'azelf', 32: 'azumarill', 33: 'azurill', 34: 'bagon', 35: 'baltoy', 36: 'banette', 37: 'barbaracle', 38: 'barboach', 39: 'basculin-red-striped', 40: 'bastiodon', 41: 'bayleef', 42: 'beartic', 43: 'beautifly', 44: 'beedrill', 45: 'beheeyem', 46: 'beldum', 47: 'bellossom', 48: 'bellsprout', 49: 'bergmite', 50: 'bewear', 51: 'bibarel', 52: 'bidoof', 53: 'binacle', 54: 'bisharp', 55: 'blacephalon', 56: 'blastoise', 57: 'blaziken', 58: 'blissey', 59: 'blitzle', 60: 'boldore', 61: 'bonsly', 62: 'bouffalant', 63: 'bounsweet', 64: 'br

In [11]:
# 실제 동작 부분 - 시간 좀 걸림..

# 첫번째 PATHS[]에 비교하고자 하는 포켓몬 번호 입력
# ex) 0번째 인덱스의 포켓몬과 0부터 809번째(전체) 인덱스의 포켓몬 비교
pokemon(PATHS[0], PATHS[0:809]) # 결과값 구할 때마다 수정해야 하는 코드

1/1 [==============================] - 0s 76ms/step
images\abra.png and images\abomasnow.png : 0.49911269545555115
1/1 [==============================] - 0s 73ms/step
images\abra.png and images\abra.png : 0.9999999403953552
1/1 [==============================] - 0s 72ms/step
images\abra.png and images\absol.png : 0.5286931991577148
1/1 [==============================] - ETA: 0s

c:\ProgramData\anaconda3\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1/1 [==============================] - 0s 78ms/step
images\abra.png and images\accelgor.png : 0.530694842338562
1/1 [==============================] - 0s 73ms/step
images\abra.png and images\aegislash-blade.png : 0.4665756821632385
1/1 [==============================] - 0s 73ms/step
images\abra.png and images\aerodactyl.png : 0.5466054677963257
1/1 [==============================] - 0s 70ms/step
images\abra.png and images\aggron.png : 0.4770786166191101
1/1 [==============================] - 0s 72ms/step
images\abra.png and images\aipom.png : 0.5213915109634399
1/1 [==============================] - 0s 74ms/step
images\abra.png and images\alakazam.png : 0.7625589370727539
1/1 [==============================] - 0s 72ms/step
images\abra.png and images\alomomola.png : 0.4890041947364807
1/1 [==============================] - 0s 74ms/step
images\abra.png and images\altaria.png : 0.5510777235031128
1/1 [==============================] - 0s 74ms/step
images\abra.png and images\amaura.png : 0

In [12]:
# 결과값 출력 - 데이터프레임

# [pokemon_list[]]에 비교하고자 하는 포켓몬 번호 입력
# ex) 0번째 인덱스의 포켓몬과 0부터 809번째(전체) 인덱스의 포켓몬 비교 결과값
pokemon_result = pd.DataFrame(result, columns = [pokemon_list[0]]) # 결과값 구할 때마다 수정해야 하는 코드

# 포켓몬 인덱스를 포켓몬 이름으로 변환
pokemon_result.index = pokemon_name

# 결과값 데이터프레임
pokemon_result

,abra
abomasnow,0.49911269545555115
abra,0.9999999403953552
absol,0.5286931991577148
accelgor,0.530694842338562
aegislash-blade,0.4665756821632385
...,...
zoroark,0.5279639363288879
zorua,0.553028404712677
zubat,0.5996825098991394
zweilous,0.6388040781021118
